# import

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
import random
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
from tensorflow.keras.utils  import plot_model
from sklearn.ensemble import RandomForestRegressor

# define

In [4]:
load_address = 'C:\\Users\\Administrator\\Desktop\\trajectory_demo\\datas\\'
n_in_h = 7
n_out_h = 1

from tensorflow.keras.layers import Dense, Lambda, dot, Activation, concatenate
from tensorflow.keras.layers import Layer

def change_shape(data, input_n, output_n):
    traject = []#Create empty list of tracks
    n_in_h = input_n
    n_out_h = output_n
    for k in range(0, len(data) - (n_in_h + n_out_h), (n_in_h + n_out_h)):
        a = []
        for j in range(k, k + (n_in_h + n_out_h)):#Loop from the kth position to k+8
            b = []#Create empty list
            if j == k:#If the first
                for i in range(3):#Add x and y to b
                    b.append(data[k][i])
            else:#If not the first
                for i in range(3):#Add x and y to b
                    b.append(data[j][i])
            a.append(b)#Put each b into a
        traject.append(a)#Put a in the empty list of trajectories
    len_in = (len(traject) * 80) // 100#Use 4/5 as the training set
    train = traject[:len_in]
    test = traject[len_in:]
    #shape(None,(in+out),3)
    # Shuffle the training data
    random.shuffle(train)

    # Split the data into x_train, y_train, x_test and y_test
    x_train = []
    y_train = []
    for i in range(0, len(train)):
        temp = []
        for k in range(n_in_h):
            temp.append(train[i][k])
        x_train.append(temp)
    for i in range(0, len(train)):
        temp = []
        for k in range(n_in_h, n_in_h + n_out_h):#（7.8）
            temp.append(train[i][k])
        y_train.append(temp)
    x_test = []
    y_test = []
    for i in range(0, len(test)):
        temp = []
        for k in range(n_in_h):
            temp.append(test[i][k])
        x_test.append(temp)
    for i in range(0, len(test)):
        temp = []
        for k in range(n_in_h, n_in_h + n_out_h):
            temp.append(test[i][k])
        y_test.append(temp)

    # Convert the data into numpy array
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)

    return x_train, y_train, x_test, y_test

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def change_to_random_data(data):
    data = data.reshape(-1,data.shape[1]*data.shape[2])
    return data

def change_back(data,size,n):
    data = data.reshape(-1,size,n)
    return data

def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

class Attention(Layer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def __call__(self, hidden_states):
        hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
        score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
        score = dot([score_first_part, h_t], [2, 1], name='attention_score')
        attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = dot([hidden_states, attention_weights], [1, 1], name='context_vector')
        pre_activation = concatenate([context_vector, h_t], name='attention_output')
        attention_vector = Dense(100, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
        return attention_vector

attention = Attention(name='attention_weight')
#steps = n//2
#--------------------Encoder--------------------#
visible = tf.keras.layers.Input(shape=(n_in_h,3))
encoder = tf.keras.layers.LSTM(100, activation='relu', return_sequences=True)(visible)
# define reconstruction decoder
decoder1 = tf.keras.layers.LSTM(100, activation='relu', return_sequences=True)(encoder)
decoder1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2))(decoder1)
encoder = attention(encoder)
# define prediction decoder
cells = tf.expand_dims(encoder,1)
decoder2 = tf.keras.layers.LSTM(100, activation='relu', return_sequences=True)(cells)
decoder2 = tf.keras.layers.Dense(2*n_out_h)(decoder2)
decoder2 = tf.reshape(decoder2, shape=[-1,n_out_h,2])
model = tf.keras.models.Model(inputs=[visible],
                              outputs=[decoder1,decoder2])
model.compile(optimizer='adam', loss='mse')

In [5]:
mae_encoder = []
mse_encoder = []
rmse_encoder = []
mape_encoder = []
smape_encoder = []
r2_encoder = []

data = np.load(load_address+"0.npy")

mean = np.mean(data,axis=0)#Calculate the mean of each column
std = np.std(data, axis=0) #Calculate the standard deviation of each column

#-----------------Normalized-------------------#
data[:,0] = (data[:,0] - mean[0]) / std[0]
data[:,1] = (data[:,1] - mean[1]) / std[1]
data[:,2] = (data[:,2] - mean[2]) / std[2]
#print(data[-1:,:])

#-----------------calculate data-----------------#
x_train,y_train,x_test,y_test = change_shape(data,n_in_h ,n_out_h)

#-------------------encoder------------------#
model.fit(x_train,[x_train[:,:,:2],y_train[:,:,:2]],epochs=500,batch_size=1000,verbose=0)
x_pre_encoder,test_pre_encoder = model.predict(x_test, verbose=0)

#-----------------Denormalization-----------------#
test_pre_encoder[:,:,:] = test_pre_encoder[:,:,:] * std[:2] + mean[:2]
y_test[:,:,:2] = y_test[:,:,:2] * std[:2] + mean[:2]

y_pre =  np.sqrt(test_pre_encoder[:,:,0]**2+test_pre_encoder[:,:,1]**2)
y_ture = np.sqrt(y_test[:,:,0]**2+y_test[:,:,1]**2)



mae_encoder.append(mae(y_ture,y_pre))
mse_encoder.append(mse(y_ture,y_pre))
rmse_encoder.append(np.sqrt(mse(y_ture,y_pre)))
mape_encoder.append(mape(y_ture,y_pre))
smape_encoder.append(smape(y_ture,y_pre))
r2_encoder.append(r2_score(y_ture,y_pre))

mae_encoder = pd.DataFrame(mae_encoder)
mse_encoder = pd.DataFrame(mse_encoder)
rmse_encoder = pd.DataFrame(rmse_encoder)
mape_encoder = pd.DataFrame(mape_encoder)
smape_encoder = pd.DataFrame(smape_encoder)
r2_encoder = pd.DataFrame(r2_encoder)
evaluate = pd.concat([mae_encoder,mse_encoder,rmse_encoder,mape_encoder,smape_encoder,r2_encoder],axis=1)
evaluate.to_csv(load_address+"LSTM_encoder_attention_evalute_{0}_{1}.csv".format(n_in_h,n_out_h))




(178, 7, 3)
(178, 1, 3)
(45, 7, 3)
(45, 1, 3)


'\nplt.plot(test_pre_encoder[:,:,0])\nplt.plot(y_test[:,:,0])\nplt.plot(test_pre_encoder[:,:,1])\nplt.plot(y_test[:,:,1])\nplt.xlabel(\'Every step of one seal\') #x轴\nplt.ylabel(\'Moving distance per hour (meter)\') #y轴\nplt.savefig("xyonesealencoder4.png", dpi=500, bbox_inches=\'tight\')\n\nplt.plot(y_pre)\nplt.plot(y_ture)\nplt.savefig("diastance_one_seal_encoder.png", dpi=2000, bbox_inches=\'tight\')\n'